<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

# 增量训练

要说什么你自己 GPT 吧 记得多放一放跑出来的结果就行

现在你帮我把 weekData 里面的数据，分成不同文件，命名就 1，2，3，4，5 这种，要求，最多1073张文件在一起，但是有的时间数据缺失了，如果缺失超过25分钟的数据，也就是5张，那也要分开文件，比如 190000 下一张是 193000，就要分开他们，直接在 weekData 里面生成 weekData/1 weekData/2 weekData/3 weekData/4 这种  
|  
记得提到, API 的问题,有一些文件缺失,所以我们把不连续的图片分隔开,成为不同的图片集,来做增量训练,我总共会训练 8周的数据  
288(第一次训练) + 16055

In [1]:
import os
import shutil
from datetime import datetime, timedelta

# 设置文件夹路径
data_folder = './weekData'
output_folder = './weekData'
max_files_per_folder = 288
max_gap_minutes = 25  # 25分钟，即5个文件

# 获取文件名列表并按时间排序
file_list = sorted(
    [f for f in os.listdir(data_folder) if f.endswith('.png')],
    key=lambda x: datetime.strptime(x.split('_')[2][:12], '%Y%m%d%H%M')  # 提取和解析时间字符串
)

# 初始化变量
current_folder_index = 5
current_batch = []
previous_time = None

# 遍历文件并进行分组
for file_name in file_list:
    # 提取当前文件的时间戳
    timestamp_str = file_name.split('_')[2][:12]  # 提取时间部分并截取到分钟
    current_time = datetime.strptime(timestamp_str, '%Y%m%d%H%M')

    # 判断是否需要新建一个文件夹
    if (
        previous_time is None or  # 如果是第一个文件
        len(current_batch) >= max_files_per_folder or  # 当前文件夹已达到最大数量
        (current_time - previous_time > timedelta(minutes=max_gap_minutes))  # 时间间隔超过最大限制
    ):
        # 如果当前文件夹非空，则保存当前批次文件到新文件夹
        if current_batch:
            target_folder = os.path.join(output_folder, str(current_folder_index))
            os.makedirs(target_folder, exist_ok=True)
            for f in current_batch:
                shutil.move(os.path.join(data_folder, f), os.path.join(target_folder, f))
            print(f"Created folder {target_folder} with {len(current_batch)} files.")

            # 更新文件夹编号
            current_folder_index += 1
            current_batch = []

    # 将当前文件添加到批次中
    current_batch.append(file_name)
    previous_time = current_time

# 将最后一批文件保存到新文件夹
if current_batch:
    target_folder = os.path.join(output_folder, str(current_folder_index))
    os.makedirs(target_folder, exist_ok=True)
    for f in current_batch:
        shutil.move(os.path.join(data_folder, f), os.path.join(target_folder, f))
    print(f"Created folder {target_folder} with {len(current_batch)} files.")


In [2]:
# import os
# import shutil

# # 父文件夹路径
# parent_folder = './weekData'

# # 遍历 1 到 9 的子文件夹
# for i in range(5, 28):
#     sub_folder = os.path.join(parent_folder, str(i))
    
#     # 检查子文件夹是否存在
#     if os.path.exists(sub_folder):
#         # 遍历子文件夹中的所有文件
#         for file_name in os.listdir(sub_folder):
#             file_path = os.path.join(sub_folder, file_name)
#             # 将文件移动到父文件夹
#             if os.path.isfile(file_path):  # 确保只移动文件，忽略文件夹
#                 shutil.move(file_path, parent_folder)
#         # 删除空的子文件夹
#         os.rmdir(sub_folder)
#     else:
#         print(f"子文件夹 {sub_folder} 不存在，跳过。")

# print("文件已成功移动到父文件夹 ./weekData。")


# 开始增量训练

In [3]:
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

def get_next_model_step(folder_path='./Models/Add/'):
    # 匹配文件名的正则表达式
    model_pattern = re.compile(r'SP480_(\d+)\.h5')
    
    # 获取文件夹中的所有文件
    files = os.listdir(folder_path)
    
    # 找到所有符合 SP480_X.h5 命名格式的文件，并提取编号
    model_numbers = []
    for file in files:
        match = model_pattern.match(file)
        if match:
            model_numbers.append(int(match.group(1)))

    # 如果找到模型文件，返回最大编号 + 1；否则返回 1
    return max(model_numbers) + 1 if model_numbers else 1

In [4]:
import os
import cv2
import numpy as np
from skimage import morphology
import matplotlib.pyplot as plt

def readPicture(times, model):
# 输入文件夹路径
    input_folder = './weekData/' + str(times)

    # 初始化列表来存放原始和处理后的图片
    original_images = []
    processed_images = []

    # 设置最小降雨面积阈值
    min_rain_area = 10  # 可根据需要调整

    # 遍历文件夹中的每张图片
    for filename in sorted(os.listdir(input_folder)):
        # 检查是否为图片文件
        if filename.endswith('.png'):
            input_path = os.path.join(input_folder, filename)
            
            # Step 1: 读取图片
            img = cv2.imread(input_path)
            if img is None:
                print(f"Error: 图片 {filename} 未成功加载，跳过此文件。")
                continue
            
            # Step 2: 灰度化处理
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Step 3: 保存未处理的灰度图像到 original_images
            original_images.append(gray_img)
            
            # Step 4: 二值化处理
            _, binary_img = cv2.threshold(gray_img, 30, 255, cv2.THRESH_BINARY)
            
            # Step 5: 去除小面积区域
            binary_img = morphology.remove_small_objects(binary_img.astype(bool), min_rain_area)
            binary_img = (binary_img * 255).astype(np.uint8)
            
            # Step 6: 应用掩码
            processed_img = cv2.bitwise_and(gray_img, gray_img, mask=binary_img)
            
            # Step 7: 归一化
            normalized_img = (processed_img / 255.0)  # 归一化到 0-1 区间
            
            # 将处理后的图像添加到 processed_images
            processed_images.append(normalized_img)

    # print("图片处理完成。`original_images` 和 `processed_images` 数据已准备好。")
    return processed_images


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ConvLSTM2D, TimeDistributed, Dropout, Reshape
from tensorflow.keras.optimizers import Adam

def setDataShape(processed_images):
    # 假设 processed_images 已经准备好，每个图像大小为 (480, 480)
    processed_images = np.array(processed_images)  # 转换成 NumPy 数组
    processed_images = processed_images[..., np.newaxis]  # 添加单通道 (480, 480) -> (480, 480, 1)

    time_steps = 12  # 输入时间步
    pred_steps = 12  # 预测时间步
    num_samples = len(processed_images) - time_steps - pred_steps + 1

    # 构建输入和输出数据
    X_train = np.array([processed_images[i:i + time_steps] for i in range(num_samples)])
    y_train = np.array([processed_images[i + time_steps:i + time_steps + pred_steps] for i in range(num_samples)])

    # print("增量训练数据形状:")
    # print("X_train shape:", X_train.shape)  # (num_samples, time_steps, 480, 480, 1)
    # print("y_train shape:", y_train.shape)  # (num_samples, pred_steps, 480, 480, 1)

    return X_train, y_train

def data_generator(processed_images, time_steps, pred_steps, batch_size):
    num_samples = len(processed_images) - time_steps - pred_steps + 1
    while True:
        for i in range(0, num_samples, batch_size):
            X = np.array([processed_images[j:j + time_steps] for j in range(i, min(i + batch_size, num_samples))], dtype=np.float32)
            y = np.array([processed_images[j + time_steps:j + time_steps + pred_steps] for j in range(i, min(i + batch_size, num_samples))], dtype=np.float32)
            yield X, y

In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from skimage.metrics import structural_similarity as ssim

def test(model):
# 文件夹路径
    test_folder = './dataOneDayTest'
    time_steps = 12
    pred_steps = 12

    # 加载测试图像数据并预处理
    test_images = []
    image_files = sorted([f for f in os.listdir(test_folder) if f.endswith('.png')])
    for file in image_files:
        img = cv2.imread(os.path.join(test_folder, file), cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (480, 480), interpolation=cv2.INTER_AREA)
        normalized_img = resized_img / 255.0  # 归一化
        test_images.append(normalized_img)

    # 转换成 NumPy 数组并添加通道维度
    test_images = np.array(test_images)[..., np.newaxis]  # 形状: (24, 480, 480, 1)

    # 准备测试输入数据
    X_test = np.array([test_images[i:i + time_steps] for i in range(len(test_images) - time_steps - pred_steps + 1)])
    y_actual = np.array([test_images[i + time_steps:i + time_steps + pred_steps] for i in range(len(test_images) - time_steps - pred_steps + 1)])

    # print("X_test shape:", X_test.shape)  # (num_samples, time_steps, 480, 480, 1)
    # print("y_actual shape:", y_actual.shape)  # (num_samples, pred_steps, 480, 480, 1)

    # 使用模型进行预测
    y_pred = model.predict(X_test)

    # 计算 MSE, MAE 和 SSIM
    mse = mean_squared_error(y_actual.flatten(), y_pred.flatten())
    mae = mean_absolute_error(y_actual.flatten(), y_pred.flatten())

    # 计算结构相似性（SSIM）
    ssim_scores = []
    for i in range(pred_steps):
        ssim_score = ssim(y_actual[0, i, :, :, 0], y_pred[0, i, :, :, 0], data_range=1.0)
        ssim_scores.append(ssim_score)
    mean_ssim = np.mean(ssim_scores)

    # 输出评估指标
    print(f"Mean Squared Error (MSE) on Test Set: {mse}")
    print(f"Mean Absolute Error (MAE) on Test Set: {mae}")
    print(f"Mean Structural Similarity (SSIM) on Test Set: {mean_ssim}")

    # 可视化前几个预测和实际结果的对比
    fig, axs = plt.subplots(12, 2, figsize=(10, 40))
    for i in range(12):
        # 显示预测结果
        axs[i, 0].imshow(y_pred[0, i, :, :, 0], cmap='gray')
        axs[i, 0].set_title(f"Predicted Frame {i+1}")
        axs[i, 0].axis('off')
        
        # 显示实际结果
        axs[i, 1].imshow(y_actual[0, i, :, :, 0], cmap='gray')
        axs[i, 1].set_title(f"Actual Frame {i+1}")
        axs[i, 1].axis('off')

    plt.tight_layout()
    plt.show()
    return mse,mae,mean_ssim


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

start_from = get_next_model_step()
print(f"下一个模型编号为：{start_from}")
max_times = 56
time_steps = 12
pred_steps = 12

# 设置早停回调
early_stopping = EarlyStopping(
    monitor='loss',
    patience=3,
    min_delta=0.0001,
    restore_best_weights=True
)
# 执行 16 次微调和测试
for times in range(start_from, max_times+1):

        
    # 加载已有的模型
    model_path = './Models/Add/SP480_'+str(times-1)+'.h5'
    model = load_model(model_path)

    # 设置增量训练参数
    learning_rate = 1e-5  # 设置较低的学习率

    # 编译模型
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    print(f"\n正在进行第 {times} 次微调和测试...")

    # 清理显存和重置模型状态
    tf.keras.backend.clear_session()

    # 读取当前批次的图片
    processed_images = readPicture(times, model)

    # 调用 setDataShape 函数获取训练数据
    X_train, y_train = setDataShape(processed_images)

    # 设置批次大小和每轮训练的步数
    batch_size = 8
    steps_per_epoch = (len(processed_images) - time_steps - pred_steps + 1) // batch_size

    history = model.fit(
        X_train, y_train,
        epochs=9,              # 最多 300 个 epoch
        batch_size=1,
        callbacks=[early_stopping]  # 使用 EarlyStopping
    )

    # 保存微调后的模型
    fine_tuned_model_path = f'./Models/Add/SP480_{times}.h5'
    model.save(fine_tuned_model_path)
    # print(f"微调后的模型已保存至 {fine_tuned_model_path}")

    # 调用测试函数，并存储结果
    # mse, mae, mean_ssim = test(model)
    # test_results[times] = [mse, mae, mean_ssim]  # 存储到数组中

    # print(f"第 {times} 次测试结果 - MSE: {mse}, MAE: {mae}, SSIM: {mean_ssim}")

# # 打印所有的测试结果
# print("\n所有测试结果：")
# for i, (mse, mae, mean_ssim) in enumerate(test_results, start=1):
#     print(f"微调后模型 {i}: MSE = {mse}, MAE = {mae}, SSIM = {mean_ssim}")


下一个模型编号为：51

正在进行第 51 次微调和测试...
Epoch 1/9
265/265 [==============================] - 60s 202ms/step - loss: 0.0025
Epoch 2/9
225/265 [========================>.....] - ETA: 8s - loss: 0.0025

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/time_distributed_3/conv2d_2/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "E:\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "E:\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "e:\study-master\NUSCampus\sem1\IND5003\code\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "e:\study-master\NUSCampus\sem1\IND5003\code\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "e:\study-master\NUSCampus\sem1\IND5003\code\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "e:\study-master\NUSCampus\sem1\IND5003\code\venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      